tgb - 11/13/2019 - Testing if rescaling the inputs changes the ability of the network to generalize.   
Architecture = From [], predict []   
Test 1 = Rephrase all water concentrations as RH  
Test 2 = Rephrase all temperature as deviations from moist adiabat  
  
tgb - 11/15/2019 - This notebook is mostly used for pre-processing

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
import tensorflow as tf
import tensorflow.math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()

/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog
[Errno 2] No such file or directory: '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM'
/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


# 1) Preprocessing

In [2]:
%cd /export/nfs0home/tbeucler/CBRAIN-CAM

/nfspool-0/home/tbeucler/CBRAIN-CAM


In [3]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_SHFnsDELT_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/01/2021 09:59:26 PM Preprocess training dataset
03/01/2021 09:59:26 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_02_01_O3_SHF_nsDELT_small.nc
03/01/2021 09:59:26 PM Reading input files
03/01/2021 09:59:26 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
03/01/2021 09:59:35 PM Crop levels
03/01/2021 09:59:35 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHF_nsDELT
Computing TBP
Computing Tden
Returning normalized SHF
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: [47]
03/01/2021 09:59:36 PM Stack and reshape dataarray
03/01/2021 09:59:37 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_02_01_O3_SHF_nsDELT_small.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity

In [8]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/01/2021 10:37:41 PM Preprocess training dataset
03/01/2021 10:37:41 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_02_01_O3_LHF_nsDELQ_small.nc
03/01/2021 10:37:41 PM Reading input files
03/01/2021 10:37:41 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
03/01/2021 10:37:42 PM Crop levels
03/01/2021 10:37:42 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHF_nsDELQ
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINITY: 1 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #159: KMP_AFFINITY: 1 packages x 1 cores/pkg x 1 threads/core (1 total cores)
OMP: Info #214: KMP_A

In [9]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
03/01/2021 10:38:29 PM Preprocess training dataset
03/01/2021 10:38:29 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_02_01_O3_LHF_nsQ_small.nc
03/01/2021 10:38:29 PM Reading input files
03/01/2021 10:38:29 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
03/01/2021 10:38:30 PM Crop levels
03/01/2021 10:38:30 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHF_nsQ
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINITY: 1 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #159: KMP_AFFINITY: 1 packages x 1 cores/pkg x 1 threads/core (1 total cores)
OMP: Info #214: KMP_AFFINIT

In [30]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_02_01_O3_QSATdeficit_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
02/25/2021 08:38:08 PM Preprocess training dataset
02/25/2021 08:38:08 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_02_01_O3_QSATdeficit_small.nc
02/25/2021 08:38:08 PM Reading input files
02/25/2021 08:38:08 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
02/25/2021 08:38:10 PM Crop levels
02/25/2021 08:38:10 PM Create stacked dataarray
var is  QSATdeficit
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINITY: 1 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #159: KMP_AFFINITY: 1 packages x 1 cores/pkg x 1 threads/core (1 total cores)
OMP: Info #214: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #171: KMP_AFFINITY: OS pro

In [13]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_input_small_data_test

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/24/2021 03:32:52 PM Preprocess training dataset
01/24/2021 03:32:52 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_small.nc
01/24/2021 03:32:52 PM Reading input files
01/24/2021 03:32:52 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
01/24/2021 03:32:53 PM Crop levels
01/24/2021 03:32:53 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: [47]
01/24/2021 03:32:54 PM Stack and reshape dataarray
01/24/2021 03:32:55 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_small.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial O

In [14]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Train_Valid.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/24/2021 03:46:36 PM Preprocess training dataset
01/24/2021 03:46:36 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_TRAIN.nc
01/24/2021 03:46:36 PM Reading input files
01/24/2021 03:46:36 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-4]-*-*.nc
01/24/2021 03:50:19 PM Crop levels
01/24/2021 03:50:19 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: []
01/24/2021 03:50:59 PM Stack and reshape dataarray
01/24/2021 03:51:35 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_TRAIN.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc 

In [30]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
01/24/2021 06:48:24 PM Preprocess training dataset
01/24/2021 06:48:24 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_TEST.nc
01/24/2021 06:48:24 PM Reading input files
01/24/2021 06:48:24 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-0[6-9]-*-*.nc
01/24/2021 06:52:06 PM Crop levels
01/24/2021 06:52:06 PM Create stacked dataarray
var is  QBP
var is  TBP
var is  O3_AQUA
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
These time steps are cut: []
01/24/2021 06:52:55 PM Stack and reshape dataarray
01/24/2021 06:53:31 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_01_24_O3_TEST.nc
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc se

In [1]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/147_POG_RHinput_FLUXoutput_test.yml

python: can't open file 'preprocessing-11132019.py': [Errno 2] No such file or directory


In [4]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/118_POG_TfromTSinput_test.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [13]:
!ls -la -h /local/Tom.Beucler/SPCAM_PHYS/1*

-rw-r--r-- 1 Tom.Beucler ls-craig  15K Nov 13 22:17 /local/Tom.Beucler/SPCAM_PHYS/100_norm.nc
-rw-r--r-- 1 Tom.Beucler ls-craig 1.8G Nov 13 22:16 /local/Tom.Beucler/SPCAM_PHYS/100_train.nc
-rw-r--r-- 1 Tom.Beucler ls-craig 1.8G Nov 13 22:16 /local/Tom.Beucler/SPCAM_PHYS/100_train_shuffle.nc
-rw-r--r-- 1 Tom.Beucler ls-craig 1.8G Nov 13 22:17 /local/Tom.Beucler/SPCAM_PHYS/100_valid.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 13 22:32 /local/Tom.Beucler/SPCAM_PHYS/101_train.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 13 22:33 /local/Tom.Beucler/SPCAM_PHYS/101_train_shuffle.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 13 22:47 /local/Tom.Beucler/SPCAM_PHYS/101_valid.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 13 23:57 /local/Tom.Beucler/SPCAM_PHYS/102_train.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 13 23:58 /local/Tom.Beucler/SPCAM_PHYS/102_train_shuffle.nc
-rw-r--r-- 1 Tom.Beucler ls-craig  22G Nov 14 00:13 /local/Tom.Beucler/SPCAM_PHYS/102_valid.nc
-rw-r--r-- 1 Tom.Beucler ls

# 2) Check the preprocessed data

In [4]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [5]:
path_QSATdeficit = path_data+'2021_02_01_NORM_O3_QSATdeficit_small.nc'

In [6]:
path_TfromNS = path_data+'2021_02_01_NORM_O3_TfromNS_small.nc'

In [7]:
path_Bcons = path_data+'2021_02_01_NORM_O3_BCONS_small.nc'

In [8]:
data_QSATdeficit = xr.open_dataset(path_QSATdeficit)
data_TfromNS = xr.open_dataset(path_TfromNS)
data_Bcons = xr.open_dataset(path_Bcons)

In [9]:
data_QSATdeficit

<xarray.Dataset>
Dimensions:           (var_names: 214, var_names_single: 11)
Coordinates:
  * var_names         (var_names) object 'QSATdeficit' 'QSATdeficit' ... 'QRS'
  * var_names_single  (var_names_single) object 'QSATdeficit' 'TBP' ... 'QRS'
Data variables:
    mean              (var_names) float64 ...
    std               (var_names) float64 ...
    min               (var_names) float64 ...
    max               (var_names) float64 ...
    std_by_var        (var_names_single) float64 ...

In [17]:
data_TfromNS

<xarray.Dataset>
Dimensions:           (var_names: 214, var_names_single: 11)
Coordinates:
  * var_names         (var_names) object 'QBP' 'QBP' 'QBP' ... 'QRS' 'QRS' 'QRS'
  * var_names_single  (var_names_single) object 'QBP' 'TfromNS' ... 'QRL' 'QRS'
Data variables:
    mean              (var_names) float64 ...
    std               (var_names) float64 ...
    min               (var_names) float64 ...
    max               (var_names) float64 ...
    std_by_var        (var_names_single) float64 ...

In [23]:
data_TfromNS['mean'].shape

(214,)

In [10]:
data_QSATdeficit['mean'][:30]

<xarray.DataArray 'mean' (var_names: 30)>
array([3.514103e-02, 5.407069e-03, 3.441649e-04, 2.163555e-03, 6.659779e-04,
       2.984441e-04, 1.824647e-04, 1.344675e-04, 9.427601e-05, 6.970832e-05,
       5.265598e-05, 3.859004e-05, 3.558415e-05, 5.230486e-05, 1.046076e-04,
       2.272342e-04, 4.811124e-04, 9.102161e-04, 1.484617e-03, 2.062735e-03,
       2.350633e-03, 1.879407e-03, 9.880273e-04, 7.757872e-04, 6.943080e-04,
       7.268760e-04, 9.064377e-04, 1.236276e-03, 1.673174e-03, 1.837612e-03])
Coordinates:
  * var_names  (var_names) object 'QSATdeficit' 'QSATdeficit' ... 'QSATdeficit'

In [24]:
data_TfromNS['mean'][30:60]

<xarray.DataArray 'mean' (var_names: 30)>
array([-59.558428, -66.212014, -81.912259, -64.22248 , -67.529307, -72.369765,
       -76.540285, -77.553016, -77.075974, -75.904117, -74.379141, -72.655088,
       -70.094929, -66.142031, -60.618139, -53.835679, -46.212485, -38.179249,
       -30.09733 , -22.926994, -17.293411, -13.378349, -10.817899,  -8.781711,
        -7.18645 ,  -5.597677,  -4.051676,  -2.564866,  -1.248037,   0.      ])
Coordinates:
  * var_names  (var_names) object 'TfromNS' 'TfromNS' ... 'TfromNS' 'TfromNS'

In [27]:
data_Bcons['mean'][30:60]

<xarray.DataArray 'mean' (var_names: 30)>
array([-7.136291, -6.417706, -5.430064, -5.115713, -4.414625, -3.694918,
       -3.03892 , -2.60646 , -2.298347, -2.003542, -1.708663, -1.408087,
       -1.128457, -0.89279 , -0.70768 , -0.561357, -0.437461, -0.318778,
       -0.19061 , -0.050186,  0.087976,  0.221134,  0.332587,  0.38927 ,
        0.420973,  0.444621,  0.461478,  0.472063,  0.480409,  0.482797])
Coordinates:
  * var_names  (var_names) object 'BCONS' 'BCONS' 'BCONS' ... 'BCONS' 'BCONS'

In [28]:
data_Bcons['std'][30:60]

<xarray.DataArray 'std' (var_names: 30)>
array([0.345221, 0.358849, 0.447796, 0.472759, 0.502322, 0.659741, 0.873119,
       0.949976, 0.899141, 0.821077, 0.721789, 0.594561, 0.458232, 0.337888,
       0.250219, 0.211724, 0.210609, 0.223081, 0.238343, 0.253022, 0.266222,
       0.283593, 0.295027, 0.296891, 0.301636, 0.305545, 0.307526, 0.308157,
       0.307485, 0.308727])
Coordinates:
  * var_names  (var_names) object 'BCONS' 'BCONS' 'BCONS' ... 'BCONS' 'BCONS'

# 3) Preprocessing for PRL paper

In [2]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-RH-test.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

In [3]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-test.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages